In [1]:
from gensim.summarization import summarize
from newspaper import Article
from model import Model
from utils import build_dict, build_dataset, batch_iter
from rouge import Rouge
import pickle
import pandas as pd
import re
import tensorflow as tf
import random

import nltk 
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.tokenize import word_tokenize

import sym_vocab
import ant_vocab

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# 지문 생성 및 문제 생성을 위한 텍스트 파일 생성 
def crawl_news(url, raw_txt_name, num_word):
    # URL 기반 크롤링, 언어는 영어로 설정
    news = Article(url, language = 'en')
    news.download()
    # HTML 파싱 적용
    news.parse()
    # 지정된 단어 갯수를 기준으로 textrank를 적용하여 파싱된 텍스트를 요약 (Rule base)
    text = summarize(news.text, word_count = num_word) # 800자 기준으로 하기
    title = news.title
    title = title.lower()
    
    title = re.sub('\([^)]*\)', '', title) # 괄호 안 제거 
    
    with open(raw_txt_name, 'w', encoding = 'utf-8') as f: # 중첩 형식 사용해보기 
        f.write(text)
    
    text = re.sub('\n', ' ', text)

    return text, title

In [3]:
def bf_create_topic(raw_txt_name, prepro_txt_name):
    # raw_txt_name = './CSAT_topic.txt'
    test_dict = {}

    with open(raw_txt_name, 'r', encoding = 'utf-8') as f:
    # print(f.readlines()) # 데이터 형식은 리스트, 리스트 안에 문자열로 들어있음.
        for idx, text in enumerate(f.readlines()):
            # print(idx, text)
            text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
        
            # 사전에 저장하기
            test_dict[idx] = text
    
    with open(prepro_txt_name, 'w', encoding = 'utf-8') as f: # 중첩 형식 사용해보기
    
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    
    return test_dict 

In [4]:
def shake_bot_topic(filename, valid_article_path):
    with open("args.pickle", "rb") as f:
        args = pickle.load(f)
    
    # 재사용 가능 
    tf.reset_default_graph()  
    
    print("셰봇이 단어 사전 불러오는 중...")
    word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
    print("셰봇이 기본 설정 값 초기화 중...")
    valid_x = build_dataset("valid", valid_article_path, word_dict, article_max_len, summary_max_len, args.toy)
    valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

    with tf.Session() as sess:
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True)
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)

        batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)

        print("셰봇이 수능 문제 생성 준비 중.. {}...".format(filename))
        for batch_x, _ in batches:
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]

            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }

            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]

            with open(filename, "w") as f: # 중첩 형식 사용해보기 
                for line in prediction_output:
                    summary = list()
                    for word in line:
                        if word == "</s>":
                            break
                        if word not in summary:
                            summary.append(word)
                    print(" ".join(summary), file=f)
                    
            #with open('./rouge_test/sys_dir/{}.txt'.format(filename), "w") as f:
                #for line in prediction_output:
                    #summary = list()
                    #for word in line:
                        #if word == "</s>":
                            #break
                        #if word not in summary:
                            #summary.append(word)
                    #print(" ".join(summary), file=f)

        print('셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. {}...'.format(filename))
        
        


In [5]:
def for_rouge_test(summy, label):
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
        
    with open(summy, 'r', encoding = 'utf-8') as f:
            
        for idx, line in enumerate(f.readlines()):
            
            scores = rouge.get_scores(line, label[idx])
            
            p_score = scores[0]["rouge-1"]["p"] 
            r_score = scores[0]["rouge-1"]["r"] 
            

            r_idx_score = (idx, r_score)            
            
            rouge_list.append(r_idx_score)
            # print(rouge_list)
            
            rouge_dict[idx] = line
            
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[-1], reverse=True)
        sort_rouge_list = sort_rouge_list[:4]
        
        question_list = []
        
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
        
   
    return question_list
            

In [6]:
def for_rouge_test_correct(summy, label):
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
        
    with open(summy, 'r', encoding = 'utf-8') as f:
            
        for idx, line in enumerate(f.readlines()):
            
            scores = rouge.get_scores(line, label[idx])
            
            p_score = scores[0]["rouge-1"]["p"] 
            r_score = scores[0]["rouge-1"]["r"] 
            
            if r_score > 0:
            
                r_idx_score = (idx, r_score)            
                
                rouge_list.append(r_idx_score)
                # print(rouge_list)
            
                rouge_dict[idx] = line
            
            
        rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[1], reverse=False)
        
        rouge_list = rouge_list[:5]
        
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[0], reverse=False)
                
        question_list = []
    
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
          
    return question_list, sort_rouge_list

In [7]:
def delete_unk(content, unk_sentence):
    
    container = []
    
    text = open(content, 'r', encoding = 'utf-8').read()
    
    tagged_list = pos_tag(word_tokenize(text))
    
    
    nnp_list = [t[0] for t in tagged_list if (t[1] == "NNP") or (t[1] == "NN")]
    
    fd_names = FreqDist(nnp_list)
    
    freq = fd_names.most_common(2)
    
    #for tagged in tagged_list:
        
        #if tagged[1] == 'NN' or 'NNP':
            
            #container.append(tagged[0]) # 단어만 추출
    
    
    
    replace_sentence = re.sub('< unk >', freq[0][0], unk_sentence)
    
    replace_sentence = re.sub('# nd', 'Several', replace_sentence)
    
    replace_sentence = re.sub('\n', '', replace_sentence)
    
    return replace_sentence
 

In [8]:
def make_wrong_question(replace_sentence): # 딱 하나만 들어가야 함.
    
    
    token = word_tokenize(replace_sentence)
    
    tagged_list = pos_tag(token)
    
    va_list = [t[0] for t in tagged_list if (t[1] == "JJ") or (t[1] == "JJR") or (t[1] == "JJS")
               or (t[1] == "RB") or (t[1] == "RBS") or (t[1] == "RBR") or (t[1] == "VB") or (t[1] == "VBD")]
    
    
    for va in va_list:
        
        para = ant_vocab.crawling_dict(va)
        
        if len(para) > 0:
            
            token[token.index(va)] = para  
            
        
    replace_sentence = ' '.join(token)
    
    replace_sentence = replace_sentence.capitalize() + "."
    
    return replace_sentence
    

In [9]:
def make_order_sentence(test_dict):
    
    # 제시 문장 
    main_set = '(A)' + ' ' + test_dict[0]
    
    # 몇개의 문장씩 나눌 것인지 결정
    
    divide_num = (int(len(test_dict)-1)) // 3
    
    print(divide_num)
    
    remain_num = (int(len(test_dict)-1)) % 3
    
    print(remain_num)
    
    if divide_num >= remain_num * 2:
        
        divide_num = int(divide_num - 1)
        
        remain_num = int(remain_num + 2)
    
    
    # 랜덤하게 배정하기 위한 리스트 형성
    
    assign_list = [int(divide_num), int(divide_num), int(remain_num)]
    random.shuffle(assign_list)
    
    # 배정받은 수만큼 문장 나누기
    
    first_set = list()
    second_set = list()
    third_set = list()
    
    for n in range(1, assign_list[0] + 1):
        first_set.append(test_dict[n].capitalize())
        
    
    for n in range(assign_list[0] + 1, assign_list[0] + assign_list[1] + 1):
        second_set.append(test_dict[n].capitalize())
        
    
    for n in range(assign_list[0] + assign_list[1] + 1, assign_list[0] + assign_list[1] + assign_list[2] + 1):
        third_set.append(test_dict[n].capitalize())
    
    
    return main_set, first_set, second_set, third_set # 아직 배열 되어 있지 않은 상황. 

In [11]:
url = 'https://www.nytimes.com/2019/11/15/us/politics/elizabeth-warren-medicare-for-all-100-days.html?action=click&module=Top%20Stories&pgtype=Homepage'
num_word = 800
raw_txt_name = 'CSAT_41.txt'
prepro_txt_name = 'prepro_41.txt'
learned_txt_name = 'long_41.txt'
num = 1

def long_41(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word, num):
    
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력
    
    #print(title)
    
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    question_list = for_rouge_test(learned_txt_name, test_dict)
    
    #print(question_list)

    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.title() + "."
        
        question_list[idx] = replace_sentence
    
    #print(question_list)
    
    title = re.sub('\n+', '', title)
    
    title = title.strip()
    
    title = title.title() + "."
    
    question_list.append(title)
    
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('[41 ~ 42] 다음 글을 읽고, 물음에 답하시오.')
    print('-----------------------------------')
    print(text) # 지문 800자
    print('\n')
    
    print('41. 윗글의 제목으로 가장 적절한 것은?')
    
    for question in question_list:
        print(question)
    
    question_list, sort_rouge_list = for_rouge_test_correct(learned_txt_name, test_dict) # 5개의 선지가 나옴
    
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == num: # 오답 선지 1개 
            
            replace_sentence = make_wrong_question(replace_sentence)
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx != num: # 정답 선지 4개
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            #print(tag)
            
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")]
            
            # print(v_list)
            
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v]
                
                if paraphrasing_list == list:
                
                    random.shuffle(paraphrasing_list)
                
                    token[token.index(v)]= paraphrasing_list[0]
                    
                elif paraphrasing_list != list:
                    
                    token[token.index(v)] = paraphrasing_list
                
            
            replace_sentence = ' '.join(token)
                
            replace_sentence = replace_sentence.capitalize() + "."
    
        question_list[idx] = replace_sentence
       
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    print('\n')
    
    print('42. 윗글에 관한 내용으로 적절하지 않은 것은?')
    
    for question in question_list:
        print(question)
    
    
    
    
    
    

if __name__ == "__main__":
    
    long_41(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_word, num)

0 ms warren offered her transition plan two weeks after she released a detailed proposal to finance a medicare for all system , at a cost of $### trillion in additional federal spending over a decade
1 ms warren put forward that plan to rebut incessant questions about whether she would raise taxes on the middle class in order to fund a sweeping new government health insurance program
2 to fund a full-scale medicare for all program , ms warren would rely on big tax increases on businesses and wealthy americans , and she said that she would not increase taxes on middle-class families by one penny her plan was met with harsh criticism from mr biden , whose campaign said it was unrealistic and required mathematical gymnastics
3 her transition plan did not come with its own detailed financing proposal , but it would cost the federal government less than an eventual medicare for all system , over all , and would be funded using some mix of the revenue sources she has already identified , acc

In [11]:
url = 'https://www.nytimes.com/2019/11/15/science/hibernation-bears-winter-health.html'
raw_txt_name = 'CSAT_43.txt'
prepro_txt_name = 'prepro_43.txt'
learned_txt_name = 'long_43.txt'
num = 1
num_word = 800

def long_43(url, raw_txt_name, prepro_txt_name, learned_txt_name, num, num_word):
    
    text, title = crawl_news(url, raw_txt_name, num_word) # url과 raw_txt_name 파일명 입력
    
    #print(title)
    
    test_dict = bf_create_topic(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
      
    shake_bot_topic(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    # shake_bot_correct(learned_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    
    # question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict) # 5개의 선지가 나옴 
    
    main_set, first_set, second_set, third_set = make_order_sentence(test_dict)
    
    # main set 가공하기
    main_set = main_set.capitalize()
    
    main_set = re.sub(' [.]', '.', main_set)
    
    main_set = re.sub(' ,', ',', main_set)
    
    main_set = re.sub('\n', '', main_set)

    # question set 가공하기
    question_list = [first_set, second_set, third_set]
    
    answer_list = [first_set, second_set, third_set]
    
    alpha_list = ['(B)', '(C)', '(D)']
    
    random.shuffle(question_list) # 선지를 랜덤하게 섞기
    
    for ix, final_question in enumerate(question_list):
        
        final_question = alpha_list[ix] + ' ' + ' '.join(final_question)
        
        final_question = re.sub(' [.]', '.', final_question)
        
        final_question = re.sub(' ,', ',', final_question)
        
        final_question = re.sub('\n', '', final_question)
        
        question_list[ix] = final_question
        
    
    # 정답
    
    for ix, ans in enumerate(answer_list):
        
        ans = ' '.join(ans)
        
        ans = re.sub(' [.]', '.', ans)
        
        ans = re.sub(' ,', ',', ans)
        
        ans = re.sub('\n', '', ans)
        
        print( '{} 번째는 {}'.format(ix+1, ans)) 
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('[43 ~ 45]. 다음 글을 읽고, 물음에 답하시오.')
    print('-----------------------------------')
    print(main_set)
    print('-----------------------------------')
    print('43. 주어진 글 (A)에 이어질 내용을 순서에 맞게 배열한 것으로 가장 적절한 것은?')
    for question in question_list:
        print(str(question))
   
    
    question_list = for_rouge_test(learned_txt_name, test_dict)
    
    #print(question_list)

    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        replace_sentence = replace_sentence.title() + "."
        
        question_list[idx] = replace_sentence
    
    #print(question_list)
    
    title = re.sub('\n+', '', title)
    
    title = title.strip()
    
    title = title.title() + "."
    
    question_list.append(title)
    
    random.shuffle(question_list)
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    print('\n')
    print('44. 다음 글의 주제로 가장 적절한 것은?')
    for question in question_list:
        print(question)
        
    question_list, sort_rouge_list = for_rouge_test_correct(learned_txt_name, test_dict) # 5개의 선지가 나옴
    
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == num: # 오답 선지 1개 
            
            replace_sentence = make_wrong_question(replace_sentence)
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx != num: # 정답 선지 4개
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            #print(tag)
            
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")]
            
            # print(v_list)
            
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v]
                
                if paraphrasing_list == list:
                
                    random.shuffle(paraphrasing_list)
                
                    token[token.index(v)]= paraphrasing_list[0]
                    
                elif paraphrasing_list != list:
                    
                    token[token.index(v)] = paraphrasing_list
                
            
            replace_sentence = ' '.join(token)
                
            replace_sentence = replace_sentence.capitalize() + "."
    
        question_list[idx] = replace_sentence
       
    number_list = ['①', '②', '③', '④', '⑤']
    
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    print('\n')
    
    print('45. 윗글에 관한 내용으로 적절하지 않은 것은?')
    
    for question in question_list:
        print(question)
    
    
    
if __name__ == "__main__":
    
    long_43(url, raw_txt_name, prepro_txt_name, learned_txt_name, num, num_word)

0 a group of researchers at washington state university published a study in communications biology in september that sought to better understand what goes on in the cells of hibernating grizzly bears
1 the university is home to the wsu bear center , the only grizzly bear research center in the united states; it houses ## bears that were either raised in captivity or relocated to the center after being identified as problem bears in the wild
2 researchers took samples from the liver , fat and muscle of six captive grizzly bears at three times during the year
3 in the lab , a team of researchers analyzed the dna to understand the changes that occur in the cells over the course of the year
4 the effect of hibernation on each tissue is different , said joanna kelley , an evolutionary biologist at washington state university and one of the paper 's authors
5 hibernation is not just as simple as hibernating and not hibernating
6 there are transitional things happening throughout the year
7 

u는 사전에 없는 단어입니다
build는 사전에 없는 단어입니다


45. 윗글에 관한 내용으로 적절하지 않은 것은?
① U # s researchers study year bears.
② U # poor health university to build bears center..
③ Dna analysis shows research.
④ U # s scientists explore hibernation for.
⑤ Hibernation now.
